<a href="https://colab.research.google.com/github/naymark/CNN-primase/blob/main/CNN_primase_3_mer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Building, interpreting and evaluating a CNN trained to find important sequence regions related to potential transcription factor binding sites on DNA sequences.**

**In this notebook we build a CNN and interpret it in order to find transcription factor binding sites on the DNA sequences.**

In [ ]:
# Install and import dependencies

In [ ]:
pip install shap # Used to extract the expected gradients

In [ ]:
pip install adabelief-tf==0.2.1 # Optimizer that was found to be most effective

In [ ]:
pip install modisco # Tool to visualize the nucleotides making up the binding sites

In [6]:
import numpy as np
from matplotlib import pyplot as plt
import shap
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, KFold
import tensorflow as tf

from tensorflow.keras import backend as K
from tensorflow.keras import Input, layers
from tensorflow.keras.models import Model, load_model, Sequential
from google.colab import drive
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorboard.plugins.hparams import api as hp
import tensorflow_probability as tfp
from scipy import stats
import datetime
from adabelief_tf import AdaBeliefOptimizer
import os
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from keras.utils import np_utils
import gc
import modisco
import modisco.visualization
from modisco.visualization import viz_sequence
import plotly.express as px
from tensorflow.keras import backend as K

In [7]:
%load_ext tensorboard
# Used to evaluate the model during training and hyperparameter tuning

In [8]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# **How the data was processed and models built:**

Here we define the functions to process the data and build the network.

This model was trained to take an input of 3-mers, created by combining all of the overlapping 3-mer regions in each sequence (positions, [0,1,2], [1,2,3], [2,3,4], and so on).
- Expected gradients were then calculated for each 3-mer, and these values were plotted, for a few sequences with a high fluorescence label, using a Plotly bar chart.

In [9]:
# Functions to import and preprocess the data, and to build and train the model
def importDataSet():
    data = pd.read_csv(r'/content/gdrive/MyDrive/Mtb.csv')
    data.drop(columns=['ID','Sequence'], inplace=True)
    data.drop(columns=data.columns[37:-1], inplace=True)
    return data

def splitDataToLabels_Features(data):
    features = data.drop(columns=['name2','F488 Median']).to_numpy()
    labels = data['F488 Median'].to_numpy()#.values
    labels=labels.reshape(len(labels),1)
    return features, labels

def encoding_features(data, sequence_type, k_mers=3):
    num_cols = data[data.columns[1:-1]].shape[1]
    encoded_data = pd.get_dummies(data, columns = data.columns[1:-1])
    if sequence_type == 'all':
      encoded_data = encoded_data
    elif sequence_type == 'de_bruijn':
      encoded_data = encoded_data[encoded_data['name2'].str.startswith('DE', na=False)]
    elif sequence_type == 'k-mers':
      encoded_data = encoded_data[encoded_data['name2'].str.startswith('4', na=False) |
                  encoded_data['name2'].str.startswith('5', na=False) |
                  encoded_data['name2'].str.startswith('6', na=False)]
    num_rows = encoded_data.shape[0]
    return data, encoded_data, num_cols, num_rows

def reshape_features(features, num_cols, num_rows):
    features = features.reshape(num_rows,num_cols,-1)
    return features

def scale_labels(train_labels, test_labels):
    minmax_sc = MinMaxScaler()
    minmax_sc.fit(train_labels)
    train_labels=minmax_sc.transform(train_labels)
    test_labels=minmax_sc.transform(test_labels)
    return train_labels, test_labels

def preprocess_data(sequence_type='all', scale_label=True):
    data = importDataSet()
    data, encoded_data, num_cols, num_rows = encoding_features(data,sequence_type)
    features,labels = splitDataToLabels_Features(encoded_data)
    features = reshape_features(features, num_cols, num_rows)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.1, random_state = 1)
    if scale_label:
      y_train, y_test = scale_labels(y_train, y_test)
    return X_train, X_test, y_train, y_test, data, features

def tf_pearson(y_true, y_pred): # Pearson used as the model's accuracy metric
    return tfp.stats.correlation(y_true, y_pred)

def build_cnn(final_units=1, final_activation=None):
    cnn = tf.keras.models.Sequential()
    cnn.add(tf.keras.layers.Conv1D(filters=512, kernel_size=3, strides=1, input_shape=[34,64], name='conv1d_layer'))
    cnn.add(tf.keras.layers.BatchNormalization())
    cnn.add(tf.keras.layers.Activation('gelu'))
    cnn.add(tf.keras.layers.GlobalMaxPooling1D(name='maxpool_layer'))
    cnn.add(tf.keras.layers.Flatten())
    cnn.add(tf.keras.layers.Dense(units=64, activation=None, name='dense_layer'))
    cnn.add(tf.keras.layers.BatchNormalization())
    cnn.add(tf.keras.layers.Activation('gelu'))
    cnn.add(tf.keras.layers.Dropout(0.2, name='dropout_layer'))
    cnn.add(tf.keras.layers.Dense(units=final_units, activation=final_activation, name='output_layer'))
    return cnn

# Create a function where the model is trained using automatic hyperparameter tuning
def train_test_hparams(hparams, final_units=1, final_activation=None):
    cnn = tf.keras.models.Sequential(build_cnn().layers[:-1])
    cnn.add(tf.keras.layers.Dense(units=hparams[HP_NUM_UNITS], activation='gelu', name='second_dense_layer'))
    cnn.add(tf.keras.layers.BatchNormalization())
    cnn.add(tf.keras.layers.Activation('gelu'))
    cnn.add(tf.keras.layers.Dropout(hparams[HP_DROPOUT]))
    cnn.add(tf.keras.layers.Dense(units=final_units, activation=final_activation, name='output_layer'))
    cnn.compile(optimizer = hparams[HP_OPTIMIZER], loss = 'mean_squared_error', metrics = ['accuracy'])
    
    # Use callbacks to enable TensorBoard logging of the parameters during training
    cnn.fit(x=X_train, 
              y=y_train, 
              epochs=3,
              batch_size=32,
              callbacks=[tensorboard_callback],
              validation_split=0.1)
    _, accuracy = cnn.evaluate(X_test, y_test)
    return accuracy, cnn 

def run_hparam_tuner(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy, cnn = train_test_hparams(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

The following lines of code are for preprocessing the data into 3-mers.




---



In [10]:
# Import data
data = importDataSet()
data.head()

,name2,P0,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,F488 Median
0,T7_00001_G01_r1_o1,C,C,A,A,A,A,A,A,A,A,C,C,A,A,C,C,C,G,T,C,A,A,A,A,C,C,C,C,A,C,A,A,C,C,C,C,17326
1,T7_00005_G01_r1_o1,A,A,A,C,C,C,A,A,A,A,A,A,A,A,A,C,C,G,T,C,C,C,A,A,C,C,A,C,C,C,A,A,C,C,C,C,16256
2,T7_00009_G01_r1_o1,C,A,A,A,A,A,A,C,C,A,A,A,A,C,C,C,C,G,T,C,A,A,C,C,C,C,A,A,C,C,C,C,A,A,A,C,21020
3,T7_00013_G01_r1_o1,C,C,A,C,C,C,A,A,A,A,A,A,A,A,A,C,C,G,T,C,A,C,C,C,C,A,A,A,C,C,A,A,A,C,C,C,18998
4,T7_00017_G01_r1_o1,A,A,C,C,A,A,C,A,C,C,C,A,A,A,A,A,C,G,T,C,C,C,A,A,C,C,A,A,C,C,C,A,C,C,A,A,29038


In [11]:
# Create 3-mers
for col in range(1,35):
      data.iloc[:,col] = data.iloc[:,col].str.cat(data.iloc[:,col+1], sep='')
      data.iloc[:,col] = data.iloc[:,col].str.cat(data.iloc[:,col+2], sep='')

In [12]:
data.head()

,name2,P0,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,F488 Median
0,T7_00001_G01_r1_o1,CCA,CAA,AAA,AAA,AAA,AAA,AAA,AAA,AAC,ACC,CCA,CAA,AAC,ACC,CCC,CCG,CGT,GTC,TCA,CAA,AAA,AAA,AAC,ACC,CCC,CCC,CCA,CAC,ACA,CAA,AAC,ACC,CCC,CCC,C,C,17326
1,T7_00005_G01_r1_o1,AAA,AAC,ACC,CCC,CCA,CAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAC,ACC,CCG,CGT,GTC,TCC,CCC,CCA,CAA,AAC,ACC,CCA,CAC,ACC,CCC,CCA,CAA,AAC,ACC,CCC,CCC,C,C,16256
2,T7_00009_G01_r1_o1,CAA,AAA,AAA,AAA,AAA,AAC,ACC,CCA,CAA,AAA,AAA,AAC,ACC,CCC,CCC,CCG,CGT,GTC,TCA,CAA,AAC,ACC,CCC,CCC,CCA,CAA,AAC,ACC,CCC,CCC,CCA,CAA,AAA,AAC,A,C,21020
3,T7_00013_G01_r1_o1,CCA,CAC,ACC,CCC,CCA,CAA,AAA,AAA,AAA,AAA,AAA,AAA,AAA,AAC,ACC,CCG,CGT,GTC,TCA,CAC,ACC,CCC,CCC,CCA,CAA,AAA,AAC,ACC,CCA,CAA,AAA,AAC,ACC,CCC,C,C,18998
4,T7_00017_G01_r1_o1,AAC,ACC,CCA,CAA,AAC,ACA,CAC,ACC,CCC,CCA,CAA,AAA,AAA,AAA,AAC,ACG,CGT,GTC,TCC,CCC,CCA,CAA,AAC,ACC,CCA,CAA,AAC,ACC,CCC,CCA,CAC,ACC,CCA,CAA,A,A,29038


In [13]:
encoded_data = pd.get_dummies(data.iloc[:,1:-3])

In [14]:
features = encoded_data.to_numpy()

In [15]:
features = features.reshape(len(data),34,-1)

In [16]:
labels = data.iloc[:len(data),-1].to_numpy()#.values
labels=labels.reshape(len(labels),1)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2, random_state = 1)

In [18]:
y_train, y_test = scale_labels(y_train, y_test)



---



# Here we create the model (that takes sequences processed as individual nucleotides) and load the data, using the functions defined above.

Note:
- We use AdaBelief as an optimizer, which is claimed by the authors of the paper (https://arxiv.org/abs/2010.07468) to train as fast as Adam and generalize as well as SGD.
- We also use Pearson correlation as a metric
- Finally, note that the debruin set alone did not train any model well; the k-mer dataset worked very well with most models but failed in some; and the entire dataset was the most resilient across models

In [19]:
# Create initial CNN and choose DNA sequence type ['all', 'k-mers', 'de_bruijn']
OUTPUT = 1 # This is the number of neurons in the last layer, 1 fits a regression problem;
          # \ if we're dealing with classifications we need num_neurons=num_categories
ACTIV = None
cnn = build_cnn(final_units=OUTPUT, final_activation=ACTIV)

# Compile the CNN
OPTIMIZER = AdaBeliefOptimizer(learning_rate=1e-2, epsilon=1e-14, rectify=False, print_change_log = False)
LOSS = tf.keras.losses.MeanSquaredError()    
METRICS = tf_pearson
cnn.compile(optimizer = OPTIMIZER, loss = LOSS, metrics = [METRICS])

SEQUENCE_TYPE = 'all'
#X_train, X_test, y_train, y_test, data, features = preprocess_data(sequence_type=SEQUENCE_TYPE, scale_label=True)

We create some callbacks to enable logging and early stopping while training the model.

In [20]:
log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stops = EarlyStopping(patience=15, monitor='val_tf_pearson', mode='max', restore_best_weights=True)
filepath = '/content/gdrive/MyDrive/cnn_model'
ckpt_callback = ModelCheckpoint(filepath=filepath,
                              monitor='val_tf_pearson', 
                              verbose=1, 
                              save_best_only=True, 
                              mode='max')

The following code is to aid in hyperparameter tuning, **prior** to training, evaluating and interpreting the best model.

In [ ]:
"""
# Train the model using hyperparameter tuning logged to TensorBoard
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([32, 16]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'rmsprop'])) # In the end we chose Adabelief (see above),
#\ because after hyperparameter tuning we got the best pearson correlation
METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )
  
session_num = 0
for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run_hparam_tuner('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1 
"""

After running the hyperparameter tuning code above, the metrics of the models using each combination of hyperparameters can be compared using Tensorboard:

In [ ]:
#!kill XXXX

In [ ]:
# Compare hyperparameter metrics
#%tensorboard --logdir logs/hparam_tuning

We train the CNN using the best hyperparameters.

In [ ]:
# Train the CNN with the chosen hyperparameters and save best model
cnn.fit(x=X_train,
              y=y_train,
              epochs=150,
              batch_size=32,
              callbacks=[tensorboard_callback, early_stops, ckpt_callback],
              validation_split=0.2)

# Save best model
filename = '/content/gdrive/MyDrive/cnn_model/cnn_model_' + SEQUENCE_TYPE + str(OUTPUT) + ('' if ACTIV == None else ACTIV)\
  + LOSS.name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
cnn.save(filename)

In [ ]:
# Fine-tune using smaller learning rate
dependencies = {'tf_pearson': tf_pearson}
OPTIMIZER = AdaBeliefOptimizer(learning_rate=1e-3, epsilon=1e-14, rectify=False, print_change_log = False)
cnn = load_model(filename, compile=False, custom_objects=dependencies)
cnn.compile(optimizer = OPTIMIZER, loss = LOSS, metrics = [METRICS])
cnn.fit(x=X_train, 
          y=y_train, 
          epochs=150,
          batch_size=32,
          callbacks=[tensorboard_callback, early_stops, ckpt_callback],
          validation_split=0.2)

# Re-save best model
cnn.save(filename)

In [ ]:
# Further fine-tune using smaller learning rate
dependencies = {'tf_pearson': tf_pearson}
OPTIMIZER = AdaBeliefOptimizer(learning_rate=1e-4, epsilon=1e-14, rectify=False, print_change_log = False)
cnn = load_model(filename, compile=False, custom_objects=dependencies)
cnn.compile(optimizer = OPTIMIZER, loss = LOSS, metrics = [METRICS])
cnn.fit(x=X_train, 
          y=y_train, 
          epochs=150,
          batch_size=32,
          callbacks=[tensorboard_callback, early_stops, ckpt_callback],
          validation_split=0.2)

# Re-save best model
cnn.save(filename)

In [ ]:
#!kill XXXX

The updated accuracy and loss of the model throughout training can be seen using Tensorboard.

In [ ]:
# Show model with histograms of the trainable parameters
%tensorboard --logdir logs/fit/

Here we load some of our pre-trained models for evaluation and interpretation.

In [21]:
dependencies = {'tf_pearson': tf_pearson}
OPTIMIZER = AdaBeliefOptimizer(learning_rate=1e-3, epsilon=1e-14, rectify=False, print_change_log = False)
# Three-mer model
cnn = load_model('/content/gdrive/MyDrive/cnn_model/cnn_model_all1mean_squared_error20210506-194646',\
                 compile=False, custom_objects=dependencies)
# Four-mer model
#cnn = load_model('/content/gdrive/MyDrive/cnn_model/cnn_model_all1mean_squared_error20210420-065811',\
#                 compile=False, custom_objects=dependencies)
# Model with 3-, 4-, 5- and 6-mer conv layers 
#cnn = load_model('/content/gdrive/MyDrive/cnn_model/cnn_model_all1mean_squared_error20210418-074354',\
#                 compile=False, custom_objects=dependencies)
cnn.compile(optimizer = OPTIMIZER, loss = LOSS, metrics = [METRICS])

Here we evaluate the CNN. The current model gives a Pearson correlation of .92 on the test set. We can now proceed to interpret what the model has learned in terms of important k-mers which may reflect transcription factor binding sites in a given sequence.

In [22]:
# Evaluate model on the test set
cnn.evaluate(X_test, y_test)

370/370 [==============================] - 33s 2ms/step - loss: 0.0091 - tf_pearson: 0.9177


[0.008883378468453884, 0.9200088977813721]

Here we extract the expected gradients of the model. Expected gradients are a combination of the integrated gradients and shapley values.

The point of this is to use the values as importance scores which we then plotted in one of the following ways:

1.   As shown in the DeepSELEX paper (https://pubmed.ncbi.nlm.nih.gov/33381817/, the importance scores can be plotted using the tf-modisco library  (https://github.com/kundajelab/tfmodisco).

2.   Alternatively, the expected gradients can be attributed to each feature of the input (whether per nucleotide or k-mer, depending on the model used), and then plotted, for example in a bar graph.



In [25]:
#Extract the expected gradients using GradientExplainer, in order to interpret the important features found by the model
#(https://shap-lrjball.readthedocs.io/en/latest/index.html)
explainer = shap.GradientExplainer(cnn, X_train)

In [26]:
predictions = cnn.predict(X_test) # Generate predictions and sort test set based on predicted label (descending)
pred_hi_idx = np.argsort(predictions, axis=0)[::-1]
X_test_sorted = X_test[pred_hi_idx[:,0]]
X_test_top_100 = X_test_sorted[:100]

In [27]:
shap_values = explainer.shap_values(X_test_top_100) # Get expected gradient values (use X_test_top_100 for top 100 test sequences)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning:

`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.



In [ ]:
minmax_sc = MinMaxScaler()

In [ ]:
shap_values_scaled = np.reshape(minmax_sc.fit_transform(np.reshape(shap_values[0], (shap_values[0].shape[0],-1))),(shap_values[0].shape[0],shap_values[0].shape[1],-1))

In [28]:
# This is when choosing not to scale the expected gradients
shap_values_scaled = shap_values[0]

In [29]:
nucs = np.unique(data.iloc[:,1:-3])

In [30]:
test_sorted_nucs = nucs[np.argmax(X_test_top_100, axis=2)]

In [31]:
test_sorted_shap = np.zeros((shap_values_scaled.shape[0],shap_values_scaled.shape[1]))
for row in range(shap_values_scaled.shape[0]):
  for col in range(shap_values_scaled.shape[1]):
    test_sorted_shap[row,col] = shap_values_scaled[row,col,np.argmax(X_test_top_100[row,col])]

In [32]:
for i in range(10):
  k_mer_values_df = pd.DataFrame(np.column_stack((test_sorted_nucs[i].T,test_sorted_shap[i].T)), columns=['K-mers', 'Gradients'])
  k_mer_values_df['Position'] = np.arange(k_mer_values_df.shape[0])
  fig = px.bar(k_mer_values_df, x='Position', y='Gradients', color='K-mers')
  fig.show()

In [33]:
top_100_seq_k_mer_values_df = pd.DataFrame(np.column_stack((test_sorted_nucs.flatten().T,test_sorted_shap.flatten().T)), columns=['K-mers', 'Gradients'])

In [34]:
top_100_seq_k_mer_values_df['Gradients'] = top_100_seq_k_mer_values_df['Gradients'].astype('float64')

Below is a list of three-mers from the top 100 test sequences, sorted by mean of the gradient.

In [35]:
top_100_seq_k_mer_values_df.groupby('K-mers').Gradients.agg(['sum','mean','count']).sort_values(['mean'], ascending=False)

,sum,mean,count
K-mers,,,
GCC,5.642037,0.035937,157
GCT,1.586332,0.029931,53
CAT,1.021109,0.029175,35
CCC,2.696651,0.028386,95
CAC,1.976513,0.027838,71
...,...,...,...
GAG,-0.310083,-0.009690,32
CGG,-0.344293,-0.010126,34
AAT,-0.167894,-0.011193,15
